In [1]:
from pylammpsmpi.lammps import LammpsLibrary

In [2]:
lmp = LammpsLibrary(cores=2)

Set up system

In [3]:
lmp.file("in.simple")

In [4]:
lmp.extract_setting("units")

-1

In [5]:
lmp.extract_atom("f")

array([[-2.55291614e-01,  4.60941920e+00,  1.14885568e+01],
       [ 1.63726610e+01, -7.16777137e+00, -1.64907384e+01],
       [ 4.15932256e+00,  3.04975634e+00, -1.67257643e+00],
       [-1.03587349e+01, -9.44903436e+00,  1.96350670e+01],
       [-9.41120525e-01,  1.26927624e+00,  8.58130095e-01],
       [ 2.93871747e+00,  1.87783224e+00,  4.39543497e+00],
       [-4.46655274e+00,  1.43132848e+01,  9.99293197e+00],
       [ 3.30093149e+00, -9.62076772e+00, -1.01793469e+01],
       [ 7.50410385e+00, -3.43840860e+00,  2.03795196e+00],
       [ 3.28791936e-01, -1.49387836e+01, -5.24161399e+00],
       [-9.40647696e-01,  8.72760125e+00, -5.07489529e+00],
       [-4.24572228e+00,  5.93518446e-01, -2.40070941e+00],
       [ 6.01846944e+00,  3.80562013e+00,  4.41557196e+00],
       [ 6.22454868e+00, -6.30698757e+00,  1.57501404e+01],
       [-1.30571909e+00,  5.95363116e-01,  1.37370072e+00],
       [-4.54408845e+00, -9.17273714e+00, -1.29791855e+01],
       [ 4.30263057e+00,  6.36508567e+00

In [6]:
lmp.gather_atoms("f")

array([[-2.55291614e-01,  4.60941920e+00,  1.14885568e+01],
       [ 1.63726610e+01, -7.16777137e+00, -1.64907384e+01],
       [ 4.15932256e+00,  3.04975634e+00, -1.67257643e+00],
       [-1.03587349e+01, -9.44903436e+00,  1.96350670e+01],
       [-9.41120525e-01,  1.26927624e+00,  8.58130095e-01],
       [ 2.93871747e+00,  1.87783224e+00,  4.39543497e+00],
       [-4.46655274e+00,  1.43132848e+01,  9.99293197e+00],
       [ 3.30093149e+00, -9.62076772e+00, -1.01793469e+01],
       [ 7.50410385e+00, -3.43840860e+00,  2.03795196e+00],
       [ 3.28791936e-01, -1.49387836e+01, -5.24161399e+00],
       [-9.40647696e-01,  8.72760125e+00, -5.07489529e+00],
       [-4.24572228e+00,  5.93518446e-01, -2.40070941e+00],
       [ 6.01846944e+00,  3.80562013e+00,  4.41557196e+00],
       [ 6.22454868e+00, -6.30698757e+00,  1.57501404e+01],
       [-1.30571909e+00,  5.95363116e-01,  1.37370072e+00],
       [-4.54408845e+00, -9.17273714e+00, -1.29791855e+01],
       [ 4.30263057e+00,  6.36508567e+00

In [7]:
lmp.extract_box()

([0.0, 0.0, 0.0],
 [6.718384765530029, 6.718384765530029, 6.718384765530029],
 0.0,
 0.0,
 0.0,
 [1, 1, 1],
 0)

Output is lo, hi, xy, yz, xz, periodicity, box change

In [4]:
x = lmp.extract_fix("2", 0, 1, 1)
x

-2.605817524153116

In [5]:
x = lmp.extract_variable("tt", "all", 0)
x

1.1298532212880312

In [8]:
x = lmp.extract_variable("fy", "all", 1)
x

KeyboardInterrupt: 

In [5]:
lmp.get_natoms()

256

In [4]:
x = lmp.extract_variable("test", "all", 0)
x

[]

In [5]:
lmp.set_variable("test", "30")

0

In [7]:
lmp.command("run 1")

In [8]:
x = lmp.extract_variable("test", "all", 0)
x

25.0

In [5]:
lmp.extract_box()

([0.0, 0.0, 0.0],
 [6.718384765530029, 6.718384765530029, 6.718384765530029],
 0.0,
 0.0,
 0.0,
 [1, 1, 1],
 0)

In [4]:
lmp.reset_box([0.0,0.0,0.0], [8.0,8.0,8.0], 0.0,0.0,0.0)

After resetting -> extract box doesnt work

In [5]:
lmp.extract_box()

BrokenPipeError: [Errno 32] Broken pipe

In [13]:
f = lmp.gather_atoms("f")

In [14]:
f[1][0] = 0

In [15]:
lmp.scatter_atoms("f", f)

In [16]:
f1 = lmp.gather_atoms("f")

In [12]:
f[0]

array([0., 0., 0.])

In [17]:
f1

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -7.16777137e+00, -1.64907384e+01],
       [ 4.15932256e+00,  3.04975634e+00, -1.67257643e+00],
       [-1.03587349e+01, -9.44903436e+00,  1.96350670e+01],
       [-9.41120525e-01,  1.26927624e+00,  8.58130095e-01],
       [ 2.93871747e+00,  1.87783224e+00,  4.39543497e+00],
       [-4.46655274e+00,  1.43132848e+01,  9.99293197e+00],
       [ 3.30093149e+00, -9.62076772e+00, -1.01793469e+01],
       [ 7.50410385e+00, -3.43840860e+00,  2.03795196e+00],
       [ 3.28791936e-01, -1.49387836e+01, -5.24161399e+00],
       [-9.40647696e-01,  8.72760125e+00, -5.07489529e+00],
       [-4.24572228e+00,  5.93518446e-01, -2.40070941e+00],
       [ 6.01846944e+00,  3.80562013e+00,  4.41557196e+00],
       [ 6.22454868e+00, -6.30698757e+00,  1.57501404e+01],
       [-1.30571909e+00,  5.95363116e-01,  1.37370072e+00],
       [-4.54408845e+00, -9.17273714e+00, -1.29791855e+01],
       [ 4.30263057e+00,  6.36508567e+00

In [18]:
f = lmp.gather_atoms("f", ids=[1,2])

In [19]:
f

array([[  0.        ,   0.        ,   0.        ],
       [  0.        ,  -7.16777137, -16.49073841]])

In [20]:
f[1][1] = -12

In [21]:
lmp.scatter_atoms("f", f, ids=[1,2])

In [22]:
f1 = lmp.gather_atoms("f", ids=[1,2])

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# extract force on single atom two different ways

f = lmp.extract_atom("f",3)
print("Force on 1 atom via extract_atom: ",f[0][0])

fx = lmp.extract_variable("fx","all",1)
print("Force on 1 atom via extract_variable:",fx[0])

In [ ]:
# use commands_string() and commands_list() to invoke more commands

strtwo = "run 10\nrun 20"
lmp.commands_string(strtwo)

cmds = ["run 10","run 20"]
lmp.commands_list(cmds)

In [ ]:
# delete all atoms
# create_atoms() to create new ones with old coords, vels
# initial thermo should be same as step 20

natoms = lmp.get_natoms()
atype = natoms*[1]

lmp.command("delete_atoms group all");
lmp.create_atoms(natoms,None,atype,x,v);
lmp.command("run 10");

In [ ]:
############
# test of new gather/scatter and box extract/reset methods
# can try this in parallel and with/without atom_modify sort enabled

lmp.command("write_dump all custom tmp.simple id type x y z fx fy fz");

x = lmp.gather_atoms("x",1,3)
f = lmp.gather_atoms("f",1,3)

if me == 0: print("Gather XF:",x[3],x[9],f[3],f[9])

ids = lmp.gather_atoms_concat("id",0,1)
x = lmp.gather_atoms_concat("x",1,3)
f = lmp.gather_atoms_concat("f",1,3)

if me == 0: print("Gather concat XF:",ids[0],ids[1],x[0],x[3],f[0],f[3])

ids = (2*ctypes.c_int)()
ids[0] = 2
ids[1] = 4
x = lmp.gather_atoms_subset("x",1,3,2,ids)
f = lmp.gather_atoms_subset("f",1,3,2,ids)

if me == 0: print("Gather subset XF:",x[0],x[3],f[0],f[3])

x[0] = -1.0
x[1] = 0.0
x[2] = 0.0
x[3] = -2.0
x[4] = 0.0
x[5] = 0.0
ids[0] = 100
ids[1] = 200
lmp.scatter_atoms_subset("x",1,3,2,ids,x)

x = lmp.gather_atoms("x",1,3)
if me == 0: print("Gather post scatter subset:",
                  x[3],x[9],x[297],x[298],x[299],x[597],x[598],x[599])

boxlo,boxhi,xy,yz,xz,periodicity,box_change = lmp.extract_box()
if me == 0: print("Box info",boxlo,boxhi,xy,yz,xz,periodicity,box_change)

lmp.reset_box([0,0,0],[10,10,8],0,0,0)

boxlo,boxhi,xy,yz,xz,periodicity,box_change = lmp.extract_box()
if me == 0: print("Box info",boxlo,boxhi,xy,yz,xz,periodicity,box_change)